In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [2]:
from crewai import Agent, Task, Crew


In [3]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()


In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)


In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)


In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool


In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()


In [8]:
from crewai_tools import BaseTool

class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"


In [9]:
sentiment_analysis_tool = SentimentAnalysisTool()


In [10]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)


In [11]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)


In [12]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True,
)


In [13]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)


 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: To begin the analysis of DeepLearningAI, I need to gather comprehensive information about the company's background, key personnel, recent milestones, and potential needs. To start, I'll search the internet for an overview of DeepLearningAI to establish a foundational understanding of the company.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company profile"} 

I enc

2024-10-28 18:16:01,172 - 13841231872 - __init__.py-__init__:369 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/connectionpool.py", line 466, in _make_request
    self._validate_conn(conn)
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/connectionpool.py", line 1095, in _validate_conn
    conn.connect()
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/connection.py", line 730, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/connection.py", line 909, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
               ^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/agents/lib/python3.11/site-packages/urllib3/util/ssl_.py", line 469, in ssl_wrap_socket
    ssl_soc

In [14]:
from IPython.display import Markdown
Markdown(result)


Subject: Empowering DeepLearningAI's Vision Through Strategic Collaboration

Dear Andrew,

I hope this message finds you well. I’m reaching out to congratulate you on DeepLearningAI’s recent strides in enhancing online learning experiences through innovative AI integration. Your commitment to transforming educational technology resonates deeply with us at CrewAI.

At CrewAI, we specialize in developing solutions that scale seamlessly and adapt to dynamic learning environments, much like the ones DeepLearningAI champions. We understand that the heart of your mission lies in not just disseminating knowledge but in crafting personalized, engaging learning journeys. Our advanced data analytics tools are designed to gain deep insights into learner behaviors, enabling content that is not only intuitive but also profoundly impactful.

Given DeepLearningAI's recent initiatives to expand course offerings and integrate cutting-edge technologies, we see a significant synergy between our scalable infrastructure and your growth objectives. Our solutions could further enhance your capabilities to handle an increasing number of users without compromising on the quality of the learning experience.

We would cherish the opportunity to demonstrate how our tools can integrate with your current systems effortlessly. We are keen on arranging a short demonstration at your convenience, showcasing real-time integrations and the immediate benefits they can bring to your platform.

Moreover, we believe that a partnership with CrewAI could serve as a catalyst for both our companies to set new benchmarks in the ed-tech space. If this is something you would be interested in exploring, shall we schedule a meeting to discuss this in detail? We are eager to align our efforts with DeepLearningAI’s vision and contribute to your remarkable journey.

Thank you for considering this potential collaboration. I look forward to the possibility of working together to redefine the boundaries of educational technology.

Best regards,

[Your Name]
Lead Sales Representative, CrewAI

[Your Contact Information]